In [1]:
import pandas as pd
import numpy as np

In [2]:
data_dir = "C:/Users/16306/Downloads/"

In [3]:
#Consider how we handle NAN if results aren't quite what we want

In [44]:
#Read in data
df = pd.read_csv("C:/Users/16306/Downloads/application_train.csv")
df.head()
print("Rows initially:", len(df))

#Pull target variable and remove from predictors
target_var = df["TARGET"]
df.drop(columns=["SK_ID_CURR", "TARGET"], inplace=True)

Rows initially: 307511


In [62]:
#One Hot Encode and create variables for categorical values with NAN
df_ = pd.get_dummies(df, dummy_na=True)
df_.head()

,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,WALLSMATERIAL_MODE_nan,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,EMERGENCYSTATE_MODE_nan
0,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637,-3648.0,-2120,...,0,0,0,0,1,0,0,1,0,0
1,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188,-1186.0,-291,...,0,0,0,0,0,0,0,1,0,0
2,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225,-4260.0,-2531,...,0,0,0,0,0,0,1,0,0,1
3,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039,-9833.0,-2437,...,0,0,0,0,0,0,1,0,0,1
4,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038,-4311.0,-3458,...,0,0,0,0,0,0,1,0,0,1


In [63]:
#Replace NAN values in numerical variables with mean value of column 
    #Consider trying median values as well
for col in df_.columns:
    df_[col] = df_[col].fillna(df_[col].mean())

In [72]:
#drop columns that sum to zero (They're all nan columns for categorical variables that don't have any nans)
temp = pd.DataFrame(df_.sum(axis=0))
df_.drop(columns = list(temp[temp[0] == 0].index), inplace = True)

In [73]:
#Consider removing variables that are highly correlated with other variables
df_corr = df_.corr()
df_corr

,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,WALLSMATERIAL_MODE_nan,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,EMERGENCYSTATE_MODE_nan
CNT_CHILDREN,1.000000,0.012882,0.002145,0.021374,-0.001826,-0.025573,0.330938,-0.239818,0.183395,-0.028019,...,-0.000709,0.001607,-0.002032,-0.020892,-0.025088,0.011036,0.036713,-0.038644,0.004525,0.037885
AMT_INCOME_TOTAL,0.012882,1.000000,0.156870,0.191657,0.159600,0.074796,0.027261,-0.064223,0.027805,0.008506,...,0.006149,0.023886,0.003886,0.032753,0.016523,-0.003369,-0.048745,0.050174,-0.002894,-0.049706
AMT_CREDIT,0.002145,0.156870,1.000000,0.770127,0.986588,0.099738,-0.055436,-0.066838,0.009621,-0.006575,...,0.007987,0.027255,0.005799,0.046644,0.009756,-0.007373,-0.055586,0.058256,-0.004308,-0.057547
AMT_ANNUITY,0.021374,0.191657,0.770127,1.000000,0.774661,0.118424,0.009445,-0.104331,0.038513,0.011268,...,0.010077,0.035318,0.005968,0.054987,0.009188,-0.007711,-0.063930,0.068144,-0.003910,-0.067511
AMT_GOODS_PRICE,-0.001826,0.159600,0.986588,0.774661,1.000000,0.103482,-0.053416,-0.064826,0.011561,-0.009262,...,0.009953,0.028299,0.005877,0.049740,0.012882,-0.009128,-0.060514,0.062849,-0.004846,-0.062050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WALLSMATERIAL_MODE_Wooden,0.011036,-0.003369,-0.007373,-0.007711,-0.009128,-0.056263,0.008758,-0.002846,0.000159,0.012702,...,-0.011554,-0.010162,-0.009710,-0.069666,-0.068843,1.000000,-0.135474,0.078964,0.273271,-0.126455
WALLSMATERIAL_MODE_nan,0.036713,-0.048745,-0.055586,-0.063930,-0.060514,-0.108545,0.019602,0.010224,0.067334,0.014851,...,-0.088204,-0.077575,-0.074123,-0.531832,-0.525545,-0.135474,1.000000,-0.924401,-0.048372,0.933422
EMERGENCYSTATE_MODE_No,-0.038644,0.050174,0.058256,0.068144,0.062849,0.103513,-0.019852,-0.010907,-0.069039,-0.015230,...,0.076786,0.073260,0.060550,0.502903,0.491544,0.078964,-0.924401,1.000000,-0.090624,-0.984943
EMERGENCYSTATE_MODE_Yes,0.004525,-0.002894,-0.004308,-0.003910,-0.004846,-0.029932,0.005434,-0.003571,-0.006273,0.006851,...,0.031648,-0.005178,0.049521,-0.039280,-0.007605,0.273271,-0.048372,-0.090624,1.000000,-0.082907


In [74]:
#Take absolute value of correlations
df_corr_abs = df_corr.abs()

In [82]:
#This is a mess, let's work on it
corr_dict = {}
for col in df_corr_abs.columns:
    corr_dict[col] = (df_corr_abs[col].sort_values(ascending = False).index[1], df_corr_abs[col].sort_values(ascending = False)[1])

highest_corrs = []
for val in corr_dict.values():
    highest_corrs.append(val[1])

sorted(highest_corrs)

[0.009706258820390544,
 0.010530906131240437,
 0.011058928656568849,
 0.011275414105436612,
 0.011466936858909002,
 0.01146984969983328,
 0.013094141708821722,
 0.014453429433199215,
 0.014611141295997464,
 0.014707845334371087,
 0.014878787777508647,
 0.015219257083813132,
 0.01587807600767065,
 0.016142001495559357,
 0.018049363474305677,
 0.019918568572964312,
 0.02242663942850155,
 0.026528840049437407,
 0.029205787940739636,
 0.03251547477822282,
 0.034458206073689,
 0.03498087394722537,
 0.03616073209148479,
 0.03674056481901469,
 0.03785482417533689,
 0.0389845350095574,
 0.04060216190954936,
 0.04084983554835195,
 0.04190139021996996,
 0.042379987828412034,
 0.04553483246007707,
 0.045909366542471054,
 0.045948970033035975,
 0.04689784270300796,
 0.051630035753355036,
 0.0544974753246741,
 0.05568055283133256,
 0.055821449221329235,
 0.058172451205915895,
 0.06096223634421087,
 0.0635192081742089,
 0.06513321547998097,
 0.06619660349783962,
 0.06762236373081022,
 0.067857475486

In [83]:
(250*249)/2

31125.0

In [55]:
df["WEEKDAY_APPR_PROCESS_START"].isna().sum()

0

In [47]:
#Zero binary columns
count = 0
col_list = []
for col in df.columns:
    if len(df[col].value_counts()) == 2:
        if df[col].dtype != np.int64:
            col_list.append(col)
            count += 1
        
col_list

['NAME_CONTRACT_TYPE',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'EMERGENCYSTATE_MODE']

In [49]:
#Zero binary columns
count = 0
col_list = []
for col in df.columns:
    if df[col].dtype != np.int64:
        col_list.append(col)
        count += 1

In [53]:
na_cat_vars = []
for col in df[col_list].columns:
    if df[col_list][col].isna().any():
        na_cat_vars.append(col)
        
na_cat_vars

['AMT_ANNUITY',
 'AMT_GOODS_PRICE',
 'NAME_TYPE_SUITE',
 'OWN_CAR_AGE',
 'OCCUPATION_TYPE',
 'CNT_FAM_MEMBERS',
 'EXT_SOURCE_1',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3',
 'APARTMENTS_AVG',
 'BASEMENTAREA_AVG',
 'YEARS_BEGINEXPLUATATION_AVG',
 'YEARS_BUILD_AVG',
 'COMMONAREA_AVG',
 'ELEVATORS_AVG',
 'ENTRANCES_AVG',
 'FLOORSMAX_AVG',
 'FLOORSMIN_AVG',
 'LANDAREA_AVG',
 'LIVINGAPARTMENTS_AVG',
 'LIVINGAREA_AVG',
 'NONLIVINGAPARTMENTS_AVG',
 'NONLIVINGAREA_AVG',
 'APARTMENTS_MODE',
 'BASEMENTAREA_MODE',
 'YEARS_BEGINEXPLUATATION_MODE',
 'YEARS_BUILD_MODE',
 'COMMONAREA_MODE',
 'ELEVATORS_MODE',
 'ENTRANCES_MODE',
 'FLOORSMAX_MODE',
 'FLOORSMIN_MODE',
 'LANDAREA_MODE',
 'LIVINGAPARTMENTS_MODE',
 'LIVINGAREA_MODE',
 'NONLIVINGAPARTMENTS_MODE',
 'NONLIVINGAREA_MODE',
 'APARTMENTS_MEDI',
 'BASEMENTAREA_MEDI',
 'YEARS_BEGINEXPLUATATION_MEDI',
 'YEARS_BUILD_MEDI',
 'COMMONAREA_MEDI',
 'ELEVATORS_MEDI',
 'ENTRANCES_MEDI',
 'FLOORSMAX_MEDI',
 'FLOORSMIN_MEDI',
 'LANDAREA_MEDI',
 'LIVINGAPARTMENTS_MEDI',